In [1]:
import json
import math
import escher
import cobra
import cobrakbase
import modelseed_escher
from cobrakbase.core import KBaseFBAModel
from modelseed_escher.core import EscherMap

cobrakbase 0.2.3


In [2]:
token = "64XQ7SABQILQWSEW3CQKZXJA63DXZBGH"
api = cobrakbase.KBaseAPI(token)

In [3]:
escher_seed = modelseed_escher.EscherManager(escher)

In [4]:
%run ../lib/kb_escher/kb_escher_app.py

In [5]:
kb_escher.object_cache['55494/31/2'].keys()

NameError: name 'kb_escher' is not defined

In [22]:
#kb_escher.get_chemical_abundance_data(kb_escher.object_cache['55494/31/6'], 'Glucose - Exponential')

'55494/88/2'

In [24]:
row_attributemapping = kb_escher.get_object_from_ref(o['row_attributemapping_ref'])

In [27]:
row_attributemapping['attributes']

[{'attribute': 'name', 'source': 'upload'},
 {'attribute': 'modelseed',
  'attribute_ont_id': 'Custom:Term',
  'attribute_ont_ref': 'KbaseOntologies/Custom',
  'source': 'ModelSEED'}]

In [15]:
params = {
        "column_size": 3,
        "report_height": 800,
        "grid_maps": [{
            "map_id": "custom",
            "user_map_id": ["55494/68/2"],
            "model_id": "55494/17/15",
            "model_compartment_id": "c0",
            "model_alias": "modelAlias",
            "object_ids": ["55494/40/12"],
            "cpd_abundance": ["55494/31/6"],
            "cpd_abundance_dataset": "Glucose - Exponential",
            "gene_expression": ["55494/27/1"],
            "gene_expression_dataset": ""
        }]
    }

In [16]:
kb_escher = KBaseEscher(params, api, escher_seed)
kb_escher.build()
#kb_escher.export('/Users/fliu/workspace/jupyter/python3/kbase/kb_escher/data/html/data/')

{'map_id': 'custom', 'user_map_id': ['55494/68/2'], 'model_id': '55494/17/15', 'model_compartment_id': 'c0', 'model_alias': 'modelAlias', 'object_ids': ['55494/40/12'], 'cpd_abundance': ['55494/31/6'], 'cpd_abundance_dataset': 'Glucose - Exponential', 'gene_expression': ['55494/27/1'], 'gene_expression_dataset': ''}
{'map_id': 'custom.AromaticAminoAcidBiosynthesisMap', 'model_id': '55494/17/15', 'cmp': 'c0', 'alias': 'modelAlias', 'rxn_data': '55494/40/12', 'cpd_data': ('55494/31/6', 'Glucose - Exponential'), 'gene_data': None, 'export_full_model': False}


In [31]:
kb_escher.warnings

[]